# Action1 文本抄袭自动检测分析:

如果你是某新闻单位工作人员（这里假设source=新华社），为了防止其他媒体抄袭你的文章，你打算做一个抄袭自动检测分析的工具     1）定义可能抄袭的文章来源     2）与原文对比定位抄袭的地方     原始数据：sqlResult.csv，共计89611篇     从数据库导出的文章，字段包括：id, author, source, content, feature, title, url     常用中文停用词：chinese_stopwords.txt

1、完成聚类算法（10points）     2、完成分类算法（10points）     3、完成相似度计算，Top-N排序（10points）     4、完成代码，结果正确（30points

In [1]:
import pandas as pd
import numpy as np
import jieba
import os

In [7]:
# 数据加载
news = pd.read_csv('sqlResult.csv',encoding='gb18030')
news

,id,author,source,content,feature,title,url
0,89617,NaN,快科技@http://www.kkj.cn/,此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""37""...",小米MIUI 9首批机型曝光：共计15款,http://www.cnbeta.com/articles/tech/623597.htm
1,89616,NaN,快科技@http://www.kkj.cn/,骁龙835作为唯一通过Windows 10桌面平台认证的ARM处理器，高通强调，不会因为只考...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""15""...",骁龙835在Windows 10上的性能表现有望改善,http://www.cnbeta.com/articles/tech/623599.htm
2,89615,NaN,快科技@http://www.kkj.cn/,此前的一加3T搭载的是3400mAh电池，DashCharge快充规格为5V/4A。\r\n...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""18""...",一加手机5细节曝光：3300mAh、充半小时用1天,http://www.cnbeta.com/articles/tech/623601.htm
3,89614,NaN,新华社,这是6月18日在葡萄牙中部大佩德罗冈地区拍摄的被森林大火烧毁的汽车。新华社记者张立云摄\r\n,"{""type"":""国际新闻"",""site"":""环球"",""commentNum"":""0"",""j...",葡森林火灾造成至少62人死亡 政府宣布进入紧急状态（组图）,http://world.huanqiu.com/hot/2017-06/10866126....
4,89613,胡淑丽_MN7479,深圳大件事,（原标题：44岁女子跑深圳约会网友被拒，暴雨中裸身奔走……）\r\n@深圳交警微博称：昨日清...,"{""type"":""新闻"",""site"":""网易热门"",""commentNum"":""978"",...",44岁女子约网友被拒暴雨中裸奔 交警为其披衣相随,http://news.163.com/17/0618/00/CN617P3Q0001875...
...,...,...,...,...,...,...,...
89606,5,邹峥,新华社,新华社照片，多伦多，2017年6月7日\n（体育）（2）冰球——国家女子冰球队海外选秀在多伦...,"{""type"":""冰球"",""site"":""新华社"",""url"":""http://home.x...",（体育）（2）冰球——国家女子冰球队海外选秀在多伦多展开,http://home.xinhua-news.com/gdsdetailxhsnew/22...
89607,4,王衡,新华社,新华社兰州6月3日电（王衡、徐丹）记者从甘肃省交通运输厅获悉，甘肃近日集中开建高速公路、普通...,"{""type"":""宏观经济"",""site"":""新华社"",""url"":""http://home...",（经济）甘肃集中开工35个重点交通建设项目,http://home.xinhua-news.com/gdsdetailxhsnew/22...
89608,3,张旌,新华社,\n\n2017年5月29日，在法国巴黎郊外的凡尔赛宫，法国总统马克龙出席新闻发布会。（新华...,"{""type"":""其它"",""site"":""新华社"",""url"":""http://home.x...",法国议会选举 马克龙有望获“压倒性胜利”,http://home.xinhua-news.com/gdsdetailxhsnew/22...
89609,2,夏文辉,新华社,\n\n2017年5月25日，在美国马萨诸塞州剑桥市，哈佛大学毕业生在毕业典礼上欢呼。（新华...,"{""type"":""其它"",""site"":""新华社"",""url"":""http://home.x...",哈佛大学为何取消这些新生入选资格？,http://home.xinhua-news.com/gdsdetailxhsnew/22...


In [10]:
news[news.content.isna()].head()

,id,author,source,content,feature,title,url
100,89517,NaN,中国证券报?中证网,NaN,"{""type"":""公司"",""site"":""中证网"",""commentNum"":""0"",""jo...",天和防务股东未来6个月内计划减持不超过480万股公司股份,http://www.cs.com.cn/ssgs/gsxw/201706/t2017062...
103,89514,NaN,中国证券报?中证网,NaN,"{""type"":""公司"",""site"":""中证网"",""commentNum"":""0"",""jo...",晶盛机电调整限制性股票回购价格,http://www.cs.com.cn/ssgs/gsxw/201706/t2017062...
997,88620,NaN,央广网,NaN,"{""type"":""时事要闻"",""site"":""参考消息"",""commentNum"":""0"",...",[主播不在家]第二季：主播陈亮体验垃圾清运,http://www.cankaoxiaoxi.com/china/20170623/214...
1273,88344,NaN,央广网,NaN,"{""type"":""IT业界"",""site"":""参考消息"",""commentNum"":""0"",...",LKK洛可可：想象力经济时代或已到来,http://www.cankaoxiaoxi.com/science/20170610/2...
1282,88335,NaN,央广网,NaN,"{""type"":""IT业界"",""site"":""参考消息"",""commentNum"":""0"",...",CES2017：京东发布两款叮咚智能音箱新品 开放Alpha平台,http://www.cankaoxiaoxi.com/science/20170610/2...


In [13]:
news = news.dropna(subset=['content'])
news.shape

(87054, 7)

In [14]:
# 加载停用词表
with open('chinese_stopwords.txt','r',encoding='utf-8') as file:
    stopwords = [i[:-1] for i in file.readlines()]
    
# 分词处理
def split_text(text):
    text = text.replace(' ','').replace('\n','')
    text2 = jieba.cut(text)
    result = ' '.join([w for w in text2 if w not in stopwords])
    return result

In [16]:
news.iloc[0].content

'此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/体验版内测，稳定版暂不受影响），以确保工程师可以集中全部精力进行系统优化工作。有人猜测这也是将精力主要用到MIUI 9的研发之中。\r\nMIUI 8去年5月发布，距今已有一年有余，也是时候更新换代了。\r\n当然，关于MIUI 9的确切信息，我们还是等待官方消息。\r\n'

In [17]:
split_text(news.iloc[0].content)

Building prefix dict from the default dictionary ...
Dumping model to file cache C:\Users\Zhou\AppData\Local\Temp\jieba.cache
Loading model cost 0.761 seconds.
Prefix dict has been built successfully.


'此外 本周 除 小米 手机 款 机型 外 机型 暂停 更新 发布 含 开发 版 体验版 内测 稳定版 暂不受 影响 确保 工程师 集中 全部 精力 进行 系统优化 工作 有人 猜测 精力 主要 用到 MIUI9 研发 之中 \r MIUI8 去年 发布 距今已有 一年 有余 更新换代 \r 当然 MIUI9 确切 信息 等待 官方消息 \r'

In [18]:
corpus = list(map(split_text,[str(i) for i in news.content]))#
corpus[0:10]

['此外 本周 除 小米 手机 款 机型 外 机型 暂停 更新 发布 含 开发 版 体验版 内测 稳定版 暂不受 影响 确保 工程师 集中 全部 精力 进行 系统优化 工作 有人 猜测 精力 主要 用到 MIUI9 研发 之中 \r MIUI8 去年 发布 距今已有 一年 有余 更新换代 \r 当然 MIUI9 确切 信息 等待 官方消息 \r',
 '骁龙 835 唯一 Windows10 桌面 平台 认证 ARM 处理器 高通 强调 不会 只 考虑 性能 屏蔽掉 小 核心 相反 正 联手 微软 找到 一种 适合 桌面 平台 兼顾 性能 功耗 完美 方案 \r 报道 微软 已经 拿到 一些 源码 Windows10 更好 理解 big little 架构 \r 资料 显示 骁龙 835 一款 集成 CPU GPU 基带 蓝牙 Wi Fi SoC 传统 Wintel 方案 节省 至少 30% PCB 空间 \r 按计划 今年 Q4 华硕 惠普 联想 首发 骁龙 835Win10 电脑 预计 均 二合一 形态 产品 \r 当然 高通 骁龙 未来 也许 见到 三星 Exynos 联发科 华为 麒麟 小米 澎湃 进入 Windows10 桌面 平台 \r',
 '此前 一加 3T 搭载 3400mAh 电池 DashCharge 快充 规格 5V 4A \r 电池 缩水 可能 刘作 虎 所说 一加 手机 市面 轻薄 大屏 旗舰 设定 \r 按照 目前 掌握 资料 一加 手机 拥有 5.5 寸 1080P 三星 AMOLED 显示屏 6G 8GBRAM 64GB 128GBROM 双 1600 万 摄像头 备货 量 惊喜 \r 京东 泄露 信息 一加 售价 xx99 应该 2799 2899 2999 \r',
 '这是 18 葡萄牙 中部 佩德罗 冈 地区 拍摄 森林 大火 烧毁 汽车 新华社 记者 张立 云摄 \r',
 '原 标题 44 岁 女子 跑 深圳 约会 网友 拒 暴雨 裸身 奔走 … … \r @ 深圳 交警 微博称 昨日 清晨 交警 发现 女子 赤裸 上身 行走 南坪 快速 期间 轻生 年头 一辅警 发现 赶紧 披上 黄衣 一路 劝说 \r 事发 \r 到底 发生 些 \r 南 记者 带 一起 还原 现场 \r 南 记者 龙岗 大队 坂田 中队 见到 辅警

In [48]:
corpus?

Type:        list
String form: ['此外 本周 除 小米 手机 款 机型 外 机型 暂停 更新 发布 含 开发 版 体验版 内测 稳定版 暂不受 影响 确保 工程师 集中 全部 精力 进行 系统优化 工作 有人 猜测 精力 主 <...>  再见 杜塞 茶茶 \\ n 熊 老师 最后 一条 朋友圈 发 丁宁 刘诗雯 赢得 女双 冠军 照片 \\ n 懂事 之前 情动 以后 长 不过 一天 留不住 算不出 流 写道 完 \\ n']
Length:      87054
Docstring:  
Built-in mutable sequence.

If no argument is given, the constructor creates a new empty list.
The argument must be an iterable if specified.


In [46]:
import pickle
#保存到文件
# with open('corpus.pkl', 'wb') as file:
#     pickle.dump(corpus, file)

# 加载文件
corpus = []
with open('corpus.pkl','rb') as file:
    corpus = pickle.load(file)

In [19]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer
# 计算corpus的TF-IDF
countvectorizer = CountVectorizer(encoding='gb18030',min_df=0.015)#min_df最小词频
tfidftransformer = TfidfTransformer()
countvector = countvectorizer.fit_transform(corpus)
tfidf = tfidftransformer.fit_transform(countvector)
print(tfidf.shape)

(87054, 884)


In [20]:
# 标记是否为自己的新闻
label = list(map(lambda source: 1 if '新华社' in str(source) else 0,news.source))
label

[0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [22]:
from sklearn.model_selection import train_test_split
# 数据集切分
X_train,X_test,y_train,y_test = train_test_split(tfidf.toarray(),label,test_size = 0.3)

In [25]:
# 朴素贝叶斯
from sklearn.naive_bayes import MultinomialNB,BaseDiscreteNB
model = MultinomialNB()
model.fit(X_train,y_train)

MultinomialNB()

In [27]:
from sklearn.metrics import accuracy_score,precision_score,recall_score
y_predict = model.predict(X_test)
print('准确率：',accuracy_score(y_test,y_predict))
print('精确率：',precision_score(y_test,y_predict))
print('召回率：',recall_score(y_test,y_predict))

准确率： 0.8861278094727572
精确率： 0.9617985579291505
召回率： 0.9101156926727973


In [32]:
# 使用模型进行风格预测
prediction = model.predict(tfidf.toarray())
labels = np.array(label)
# compare_new_index有两列，prediction为预测，labels为真实
compare_new_index = pd.DataFrame({'prediction':prediction,'labels':label})
copy_news_index = compare_new_index[(compare_new_index['prediction']==1)&(compare_new_index['labels']==0)].index

In [33]:
# 实际为新华社的新闻
xinhuashe_news_index = compare_new_index[compare_new_index['labels']==1].index
print('可能为copy的新闻条数',len(copy_news_index))

可能为copy的新闻条数 2810


In [34]:
from sklearn.preprocessing import Normalizer
from sklearn.cluster import KMeans
normalizer = Normalizer()
# 余弦距离映射成欧氏距离
scaled_array = normalizer.fit_transform(tfidf.toarray())

# 使用Kmeans进行全量文档进行聚类
kmeans = KMeans(n_clusters=25)
k_labels = kmeans.fit_predict(scaled_array)
k_labels.shape

(87054,)

In [35]:
# 创建id_class
id_class = {index:class_ for index,class_ in enumerate(k_labels)}
from collections import defaultdict # 没有值可以自动填默认值
class_id = defaultdict(set)
for index,class_ in id_class.items():
    #只统计新华社发布的的class_id
    if index in xinhuashe_news_index.tolist():
        class_id[class_].add(index)

In [41]:
from sklearn.metrics.pairwise import cosine_similarity
# 查找相似的文章
def find_similar_text(cpindex,top=10):
    # 只在新华社发布的文章中查找
    dist_dict = {i:cosine_similarity(tfidf[cpindex],tfidf[i]) for i in class_id[id_class[cpindex]]}
    # 从大到小进行排序
    return sorted(dist_dict.items(),key=lambda x:x[1][0],reverse=True)[:top]

In [42]:
cpindex = 3352
print('是否在新华社',cpindex in xinhuashe_news_index)
print('是否在copy_news',cpindex in copy_news_index)

是否在新华社 False
是否在copy_news True


In [43]:
similar_list = find_similar_text(cpindex)
print(similar_list)

[(3134, array([[0.96849134]])), (63511, array([[0.94643198]])), (29441, array([[0.94283416]])), (3218, array([[0.87621892]])), (29615, array([[0.86936328]])), (29888, array([[0.86215862]])), (64046, array([[0.85278235]])), (29777, array([[0.84875422]])), (63974, array([[0.73415212]])), (63975, array([[0.73415212]]))]


In [44]:
print('怀疑抄袭：\n',news.iloc[cpindex].content)
# 找一篇相似的原文
similar2 = similar_list[0][0]
print('相似的原文：\n',news.iloc[similar2].content)

怀疑抄袭：
 　　中国5月份56座城市新建商品住宅价格环比上涨，4月份为58座上涨。5月份15个一线和热点二线城市房地产市场基本稳定，5月份房地产调控政策效果继续显现。
　　统计局：15个一线和热点二线城市房价同比涨幅全部回落
　　国家统计局城市司高级统计师刘建伟解读5月份房价数据
　　5月份一二线城市房价平均涨幅继续回落
　　国家统计局今日发布了2017年5月份70个大中城市住宅销售价格统计数据。对此，国家统计局城市司高级统计师刘建伟进行了解读。
　　一、15个一线和热点二线城市新建商品住宅价格同比涨幅全部回落、9个城市环比下降或持平
　　5月份，因地制宜、因城施策的房地产调控政策效果继续显现，15个一线和热点二线城市房地产市场基本稳定。从同比看，15个城市新建商品住宅价格涨幅均比上月回落，回落幅度在0.5至6.4个百分点之间。从环比看，9个城市新建商品住宅价格下降或持平；5个城市涨幅在0.5%以内。
　　二、70个大中城市中一二线城市房价同比涨幅持续回落
　　5月份，70个城市中新建商品住宅和二手住宅价格同比涨幅比上月回落的城市分别有29和18个。其中，一二线城市同比涨幅回落尤其明显。据测算，一线城市新建商品住宅和二手住宅价格同比涨幅均连续8个月回落，5月份比4月份分别回落2.2和1.7个百分点；二线城市新建商品住宅和二手住宅价格同比涨幅分别连续6个月和4个月回落，5月份比4月份分别回落0.8和0.5个百分点。
　　三、70个大中城市中房价环比下降及涨幅回落城市个数均有所增加
　　5月份，70个城市中新建商品住宅价格环比下降的城市有9个，比上月增加1个；涨幅回落的城市有26个，比上月增加3个。二手住宅价格环比下降的城市有7个，比上月增加2个；涨幅回落的城市有30个，比上月增加8个。

相似的原文：
 　　国家统计局19日发布数据，5月份，15个一线和热点二线城市新建商品住宅价格同比涨幅全部回落，其中9个城市环比下降或持平。这9个价格环比下降或持平的城市为：北京、上海、南京、杭州、合肥、福州、郑州、深圳、成都。
　　“5月份，因地制宜、因城施策的房地产调控政策效果继续显现，15个一线和热点二线城市房地产市场基本稳定。”国家统计局城市司高级统计师刘建伟说，从同比看，15个城市新建商品住宅价格涨幅均比上月回落，回落幅度在0.5至6.4个百分点之间。从环比看，9个城市新